# Problem Identification:
Our project is concerned with classifying phishing vs non-phishing emails accurately. Given the nature of our datasets and task, we will be conducting Supervised Learning (Classification).

# Importing Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import os

# Data Cleaning

#### **Datasets Used**:

- **Figshare: Seven Phishing Email Datasets**  
  Link: [https://figshare.com/articles/dataset/Seven_Phishing_Email_Datasets/25432108](https://figshare.com/articles/dataset/Seven_Phishing_Email_Datasets/25432108)  
  Subsets: Assassin, Ling, Enron, TREC05, TREC06, TREC07, CEAS08  

- **Kaggle: Phishing Email Dataset**  
  Link: [https://www.kaggle.com/datasets/naserabdullahalam/phishing-email-dataset](https://www.kaggle.com/datasets/naserabdullahalam/phishing-email-dataset)  
  Subsets: Nazario, Nigerian_Fraud  

- **Kaggle: Phishing Email Data by Type**  
  Link: [https://www.kaggle.com/datasets/charlottehall/phishing-email-data-by-type](https://www.kaggle.com/datasets/charlottehall/phishing-email-data-by-type)  
  Subset: phishing_data_by_type  

- **Kaggle: Human & LLM Generated Emails**  
  Link: [https://www.kaggle.com/datasets/francescogreco97/human-llm-generated-phishing-legitimate-emails](https://www.kaggle.com/datasets/francescogreco97/human-llm-generated-phishing-legitimate-emails)  
  Subsets: legit, phishing  

- **Kaggle: Phishing Persuasion Dataset**  
  Link: [https://www.kaggle.com/datasets/ahmadtijjani/phishing-urgency-authority-persuasion](https://www.kaggle.com/datasets/ahmadtijjani/phishing-urgency-authority-persuasion)  
  Subset: phishing_dataset_with_category  


#### Load the Datasets into Dataframes:

In [2]:
assassin_df = pd.read_csv('../datasets/Assassin.csv', low_memory=False)
ceas_df = pd.read_csv('../datasets/CEAS_08.csv', low_memory=False)
enron_df = pd.read_csv('../datasets/Enron.csv', low_memory=False)
ling_df = pd.read_csv('../datasets/Ling.csv', low_memory=False)
trec05_df = pd.read_csv('../datasets/TREC_05.csv', engine='python', on_bad_lines='skip')
trec06_df = pd.read_csv('../datasets/TREC_06.csv', engine='python', on_bad_lines='skip')
trec07_df = pd.read_csv('../datasets/TREC_07.csv', engine='python', on_bad_lines='skip')
nazario_df = pd.read_csv('../datasets/Nazario.csv')
nigerian_df = pd.read_csv('../datasets/Nigerian_Fraud.csv')
phishingtype_df = pd.read_csv('../datasets/phishing_data_by_type.csv')
legit_df = pd.read_csv('../datasets/legit.csv')
phishing_df = pd.read_csv('../datasets/phishing.csv', engine='python', on_bad_lines='skip')
phishingcatagory_df = pd.read_csv('../datasets/phishing_dataset_with_category.csv')

We did engine='python' and on_bad_lines='skip' because there may be some malformed lines and this might help us to read the data better.

### Dataset 1: Assassin.csv

First things first, we will be observing what the Dataset looks like to get a feel for what we are working with.

In [3]:
assassin_df.info()
assassin_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5809 entries, 0 to 5808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sender    5809 non-null   object
 1   receiver  5599 non-null   object
 2   date      5809 non-null   object
 3   subject   5793 non-null   object
 4   body      5808 non-null   object
 5   label     5809 non-null   int64 
 6   urls      5809 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 317.8+ KB


,sender,receiver,date,subject,body,label,urls
0,Robert Elz <kre@munnari.OZ.AU>,Chris Garrigues <cwg-dated-1030377287.06fa6d@D...,"Thu, 22 Aug 2002 18:26:25 +0700",Re: New Sequences Window,"Date: Wed, 21 Aug 2002 10:54:46 -0500 ...",0,1
1,Steve Burt <Steve_Burt@cursor-system.com>,"""'zzzzteana@yahoogroups.com'"" <zzzzteana@yahoo...","Thu, 22 Aug 2002 12:46:18 +0100",[zzzzteana] RE: Alexander,"Martin A posted:\nTassos Papadopoulos, the Gre...",0,1
2,"""Tim Chapman"" <timc@2ubh.com>",zzzzteana <zzzzteana@yahoogroups.com>,"Thu, 22 Aug 2002 13:52:38 +0100",[zzzzteana] Moscow bomber,Man Threatens Explosion In Moscow \n\nThursday...,0,1
3,Monty Solomon <monty@roscom.com>,undisclosed-recipient: ;,"Thu, 22 Aug 2002 09:15:25 -0400",[IRR] Klez: The Virus That Won't Die,Klez: The Virus That Won't Die\n \nAlready the...,0,1
4,Stewart Smith <Stewart.Smith@ee.ed.ac.uk>,zzzzteana@yahoogroups.com,"Thu, 22 Aug 2002 14:38:22 +0100",Re: [zzzzteana] Nothing like mama used to make,"> in adding cream to spaghetti carbonara, whi...",0,1


From this we understand that this dataset has 5809 rows of data and 7 columns.

We will most probably drop the sender and reciever columns later on as these are unique identifiers, and don't want to train the model on this.

It seems there are around 16 rows which do not have a subject and 1 row that doesn't have a body. This is fine as sometimes emails do not have this anyway, so perhaps there may not be any harm to include such rows. 


In [4]:
print("Number of Duplicate Rows: ",assassin_df.duplicated().sum(),'\n')
print(assassin_df[assassin_df.duplicated('body')])

Number of Duplicate Rows:  0 

Empty DataFrame
Columns: [sender, receiver, date, subject, body, label, urls]
Index: []


Additionally, there do not seem to be duplicate rows which is also a plus. We can do more thorough checking by searching for duplicates in the body area only, actually this may be better and it seems there are still no duplicates.

In [5]:
assassin_df.nunique()

sender      2523
receiver    1598
date        5557
subject     4187
body        5808
label          2
urls           2
dtype: int64

From this we can understand that the 'label' and 'urls' columns are formatted in 0 and 1 which is what we desire, hence no need to normalize the labels as there are only 2 to begin with. Interestingly, the subject has lower number of unique values than the body, indicating that the same subject may have been used for different emails.

In [6]:
assassin_df.rename(columns={'label': 'isPhishing'}, inplace=True)

Here, we have fixed the column name of 'label' to 'isPhishing' for clarity and consistency through the datasets.

In [7]:
assassin_df['date'] = pd.to_datetime(assassin_df['date'].str.extract(r'(\w{3}, \d{1,2} \w{3} \d{4})')[0], format='%a, %d %b %Y',
    errors='coerce'
)

assassin_df.info()
assassin_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5809 entries, 0 to 5808
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   sender      5809 non-null   object        
 1   receiver    5599 non-null   object        
 2   date        5355 non-null   datetime64[ns]
 3   subject     5793 non-null   object        
 4   body        5808 non-null   object        
 5   isPhishing  5809 non-null   int64         
 6   urls        5809 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 317.8+ KB


,sender,receiver,date,subject,body,isPhishing,urls
0,Robert Elz <kre@munnari.OZ.AU>,Chris Garrigues <cwg-dated-1030377287.06fa6d@D...,2002-08-22,Re: New Sequences Window,"Date: Wed, 21 Aug 2002 10:54:46 -0500 ...",0,1
1,Steve Burt <Steve_Burt@cursor-system.com>,"""'zzzzteana@yahoogroups.com'"" <zzzzteana@yahoo...",2002-08-22,[zzzzteana] RE: Alexander,"Martin A posted:\nTassos Papadopoulos, the Gre...",0,1
2,"""Tim Chapman"" <timc@2ubh.com>",zzzzteana <zzzzteana@yahoogroups.com>,2002-08-22,[zzzzteana] Moscow bomber,Man Threatens Explosion In Moscow \n\nThursday...,0,1
3,Monty Solomon <monty@roscom.com>,undisclosed-recipient: ;,2002-08-22,[IRR] Klez: The Virus That Won't Die,Klez: The Virus That Won't Die\n \nAlready the...,0,1
4,Stewart Smith <Stewart.Smith@ee.ed.ac.uk>,zzzzteana@yahoogroups.com,2002-08-22,Re: [zzzzteana] Nothing like mama used to make,"> in adding cream to spaghetti carbonara, whi...",0,1


I have converted the date strings/object to a datetime object instead for easier handling. Perhaps, this can help us when splitting the dataset in a timewise fashion.

In [8]:
assassin_df.drop(columns=['sender','receiver'], inplace=True)
assassin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5809 entries, 0 to 5808
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        5355 non-null   datetime64[ns]
 1   subject     5793 non-null   object        
 2   body        5808 non-null   object        
 3   isPhishing  5809 non-null   int64         
 4   urls        5809 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 227.0+ KB


### Dataset 2: Ling.csv

### Dataset 3: Enron.csv

### Dataset 4: TREC_05.csv

### Dataset 5: TREC_06.csv

### Dataset 6: TREC_07.csv

### Dataset 7: CEAS_08.csv

### Dataset 8: Nazario.csv

In [9]:
nazario_df.info()
nazario_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1565 entries, 0 to 1564
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sender    1565 non-null   object
 1   receiver  1469 non-null   object
 2   date      1564 non-null   object
 3   subject   1561 non-null   object
 4   body      1565 non-null   object
 5   urls      1565 non-null   int64 
 6   label     1565 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 85.7+ KB


,sender,receiver,date,subject,body,urls,label
0,Mail System Internal Data <MAILER-DAEMON@monke...,NaN,28 Sep 2017 09:57:25 -0400,DON'T DELETE THIS MESSAGE -- FOLDER INTERNAL DATA,This text is part of the internal format of yo...,1,1
1,cPanel <service@cpanel.com>,jose@monkey.org,"Fri, 30 Oct 2015 00:00:48 -0500",Verify Your Account,Business with \t\t\t\t\t\t\t\tcPanel & WHM \t...,1,1
2,Microsoft Outlook <recepcao@unimedceara.com.br>,NaN,"Fri, 30 Oct 2015 06:21:59 -0300 (BRT)",Helpdesk Mailbox Alert!!!,Your two incoming mails were placed on pending...,1,1
3,Ann Garcia <AnGarcia@mcoe.org>,"""info@maaaaa.org"" <info@maaaaa.org>","Fri, 30 Oct 2015 14:54:33 +0000",IT-Service Help Desk,Password will expire in 3 days. Click Here To ...,0,1
4,"""USAA"" <usaaacctupdate@sccu4u.com>",Recipients <usaaacctupdate@sccu4u.com>,"Fri, 30 Oct 2015 14:02:33 -0500",Final USAA Reminder - Update Your Account Now,"To ensure delivery to your inbox, please add U...",1,1


This dataset has 1565 rows of data and 7 columns. There are mostly non-null values, which is what we want.

In [10]:
print("Number of Duplicate Rows: ",nazario_df.duplicated().sum(),'\n')
print(nazario_df[nazario_df.duplicated('body')])

Number of Duplicate Rows:  0 

Empty DataFrame
Columns: [sender, receiver, date, subject, body, urls, label]
Index: []


Everything seems to be a unique row of data has we have checked for duplicated in our points of interest (body and overall rows).

In [11]:
nazario_df.nunique()

sender      1438
receiver     356
date        1564
subject     1419
body        1565
urls           2
label          1
dtype: int64

Interestingly, this means that all the emails we have here are classified as phishing.
Subject lines tend to be repeated but the content within the emails (body) seems to still be different.

In [12]:
nazario_df.rename(columns={'label': 'isPhishing'}, inplace=True)

In [13]:
nazario_df['date'] = pd.to_datetime(nazario_df['date'].str.extract(r'(\w{3}, \d{1,2} \w{3} \d{4})')[0], format='%a, %d %b %Y')

In [15]:
nazario_df.info()
nazario_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1565 entries, 0 to 1564
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   sender      1565 non-null   object        
 1   receiver    1469 non-null   object        
 2   date        1140 non-null   datetime64[ns]
 3   subject     1561 non-null   object        
 4   body        1565 non-null   object        
 5   urls        1565 non-null   int64         
 6   isPhishing  1565 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 85.7+ KB


,sender,receiver,date,subject,body,urls,isPhishing
0,Mail System Internal Data <MAILER-DAEMON@monke...,NaN,NaT,DON'T DELETE THIS MESSAGE -- FOLDER INTERNAL DATA,This text is part of the internal format of yo...,1,1
1,cPanel <service@cpanel.com>,jose@monkey.org,2015-10-30,Verify Your Account,Business with \t\t\t\t\t\t\t\tcPanel & WHM \t...,1,1
2,Microsoft Outlook <recepcao@unimedceara.com.br>,NaN,2015-10-30,Helpdesk Mailbox Alert!!!,Your two incoming mails were placed on pending...,1,1
3,Ann Garcia <AnGarcia@mcoe.org>,"""info@maaaaa.org"" <info@maaaaa.org>",2015-10-30,IT-Service Help Desk,Password will expire in 3 days. Click Here To ...,0,1
4,"""USAA"" <usaaacctupdate@sccu4u.com>",Recipients <usaaacctupdate@sccu4u.com>,2015-10-30,Final USAA Reminder - Update Your Account Now,"To ensure delivery to your inbox, please add U...",1,1


In [16]:
nazario_df.drop(columns=['sender','receiver'], inplace=True)
nazario_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1565 entries, 0 to 1564
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        1140 non-null   datetime64[ns]
 1   subject     1561 non-null   object        
 2   body        1565 non-null   object        
 3   urls        1565 non-null   int64         
 4   isPhishing  1565 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 61.3+ KB


### Dataset 9: Nigerian_Fraud.csv

### Dataset 10: phishing_data_by_type.csv

### Dataset 11: legit.csv

In [22]:
legit_df.info()
legit_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1000 non-null   object
 1   label   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


,text,label
0,"Dear Michael, I hope this message finds you we...",1
1,"Dear Jennifer, We hope you're doing well. We'r...",1
2,"Dear Robert, Your attention is urgently requir...",1
3,"Dear Emily, We're writing to remind you of the...",1
4,"Dear William, We need your immediate attention...",1


In [29]:
print("Number of Duplicate Rows: ",legit_df.duplicated().sum(),'\n')
print(legit_df[legit_df.duplicated('text')])

Number of Duplicate Rows:  2 

                                                  text  label
387  Dear Michael, I hope this message finds you we...      1
388  Dear Sarah, I trust this email finds you well....      1


This is quite odd as these entries are clearly different but for some reason marked as a duplicate?

In [30]:
legit_df.nunique()

text     998
label      1
dtype: int64

label is as expected and so is text (mostly), save for the other 2 that we got above...

In [32]:
legit_df.rename(columns={'label': 'isPhishing','text': 'body'}, inplace=True)
legit_df.info()
legit_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   body        1000 non-null   object
 1   isPhishing  1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


,body,isPhishing
0,"Dear Michael, I hope this message finds you we...",1
1,"Dear Jennifer, We hope you're doing well. We'r...",1
2,"Dear Robert, Your attention is urgently requir...",1
3,"Dear Emily, We're writing to remind you of the...",1
4,"Dear William, We need your immediate attention...",1


### Dataset 12: phishing.csv

### Dataset 13: phishing_dataset_with_category.csv

### Final Grouping/Merging:

# Data Preparation